# Exotic Options Pricing Models

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# Importing Libraries
import pandas as pd
import datetime as dt
import pytz
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as si
import math
import networkx as nx

# Importing specific modules and functions
from datetime import datetime, timezone, date, time
from math import trunc
from dateutil.parser import parse
from scipy.stats import norm

## Implementing Exotic Options Pricing Models in Python

#### Pricing Models For:
#### Barrier Options, Asian Options, Binary Options, Lookback Options, Rainbow Options, and Chooser Options.

In [2]:
class mb_Option:
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity, option_type="call"):
        self.S = spot_price
        self.K = strike_price
        self.r = risk_free_rate
        self.sigma = volatility
        self.T = maturity
        self.option_type = option_type.lower()

class mb_BarrierOption(mb_Option):
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity, barrier_level, option_type="call"):
        super().__init__(spot_price, strike_price, risk_free_rate, volatility, maturity, option_type)
        self.B = barrier_level

    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(int(self.T * 252)):  # Assuming 252 trading days in a year
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            if self.option_type == "call":
                payoff += max(0, max(price_path) - self.K) if max(price_path) > self.B else 0
            else:
                payoff += max(0, self.K - min(price_path)) if min(price_path) < self.B else 0
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_AsianOption(mb_Option):
    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(int(self.T * 252)):  # Assuming 252 trading days in a year
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            geometric_average_price = np.exp(np.mean(np.log(price_path)))
            if self.option_type == "call":
                payoff += max(0, geometric_average_price - self.K)
            else:
                payoff += max(0, self.K - geometric_average_price)
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_BinaryOption(mb_Option):
    def mb_price_option(self, option_type="cash-or-nothing"):
        steps = 100
        dt = self.T / steps
        u = np.exp(self.sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(self.r * dt) - d) / (u - d)
        
        option_prices = np.zeros(steps + 1)
        for i in range(steps + 1):
            if option_type == "cash-or-nothing":
                if self.option_type == "call":
                    option_prices[i] = 1 if (self.S * u**i * d**(steps - i)) > self.K else 0
                else:
                    option_prices[i] = 1 if (self.S * u**i * d**(steps - i)) < self.K else 0
            elif option_type == "asset-or-nothing":
                if self.option_type == "call":
                    option_prices[i] = self.S * u**i * d**(steps - i) if (self.S * u**i * d**(steps - i)) > self.K else 0
                else:
                    option_prices[i] = self.S * u**i * d**(steps - i) if (self.S * u**i * d**(steps - i)) < self.K else 0
        
        for j in range(steps - 1, -1, -1):
            for i in range(j + 1):
                option_prices[i] = np.exp(-self.r * dt) * (p * option_prices[i + 1] + (1 - p) * option_prices[i])
        
        return option_prices[0]

class mb_LookbackOption(mb_Option):
    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path = [self.S]
            for _ in range(int(self.T * 252)):  # Assuming 252 trading days in a year
                price_path.append(
                    price_path[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * np.random.standard_normal()))
            if self.option_type == "call":
                payoff += max(0, max(price_path) - self.K)
            else:
                payoff += max(0, self.K - min(price_path))
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_RainbowOption(mb_Option):
    def __init__(self, spot_price1, spot_price2, strike_price, risk_free_rate, volatility1, volatility2, maturity, correlation, option_type="call"):
        super().__init__(spot_price1, strike_price, risk_free_rate, volatility1, maturity, option_type)
        self.S2 = spot_price2
        self.sigma2 = volatility2
        self.rho = correlation

    def mb_price_option(self, num_sims=10000):
        payoff = 0
        for _ in range(num_sims):
            price_path1 = [self.S]
            price_path2 = [self.S2]
            for _ in range(int(self.T * 252)):  # Assuming 252 trading days in a year
                Z1 = np.random.standard_normal()
                Z2 = self.rho * Z1 + np.sqrt(1 - self.rho**2) * np.random.standard_normal()
                price_path1.append(price_path1[-1] * np.exp((self.r - 0.5 * self.sigma**2) + self.sigma * Z1))
                price_path2.append(price_path2[-1] * np.exp((self.r - 0.5 * self.sigma2**2) + self.sigma2 * Z2))
            if self.option_type == "call":
                payoff += max(0, max(price_path1[-1], price_path2[-1]) - self.K)
            else:
                payoff += max(0, self.K - min(price_path1[-1], price_path2[-1]))
        return np.exp(-self.r * self.T) * (payoff / num_sims)

class mb_ChooserOption(mb_Option):
    def __init__(self, spot_price, strike_price, risk_free_rate, volatility, maturity, choosing_time, option_type="call"):
        super().__init__(spot_price, strike_price, risk_free_rate, volatility, maturity, option_type)
        self.t = choosing_time

    def mb_price_option(self):
        steps = 100
        dt = self.t / steps
        u = np.exp(self.sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(self.r * dt) - d) / (u - d)
        
        call_prices = np.zeros(steps + 1)
        put_prices = np.zeros(steps + 1)
        for i in range(steps + 1):
            call_prices[i] = max(0, (self.S * u**i * d**(steps - i)) - self.K)
            put_prices[i] = max(0, self.K - (self.S * u**i * d**(steps - i)))
        
        for j in range(steps - 1, -1, -1):
            for i in range(j + 1):
                call_prices[i] = np.exp(-self.r * dt) * (p * call_prices[i + 1] + (1 - p) * call_prices[i])
                put_prices[i] = np.exp(-self.r * dt) * (p * put_prices[i + 1] + (1 - p) * put_prices[i])
        
        return max(call_prices[0], put_prices[0])

### Exotic Options Pricing

In [3]:
# Initialize the parameters
spot_price = 150
strike_price = 155
risk_free_rate = 0.01
volatility = 0.20
maturity = 1
barrier_level = 160
choosing_time = 0.5
spot_price2 = 145  # For Rainbow Option
volatility2 = 0.25  # For Rainbow Option
correlation = 0.5  # For Rainbow Option

# Create instances for each type of option and calculate the price for calls and puts
# Barrier Options
barrier_call = mb_BarrierOption(spot_price, strike_price, risk_free_rate, volatility, maturity, barrier_level, "call")
barrier_put = mb_BarrierOption(spot_price, strike_price, risk_free_rate, volatility, maturity, barrier_level, "put")

print(f"Barrier Call Option Price: {barrier_call.mb_price_option():.2f}")
print(f"Barrier Put Option Price: {barrier_put.mb_price_option():.2f}")

# Asian Options
asian_call = mb_AsianOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "call")
asian_put = mb_AsianOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "put")

print(f"Asian Call Option Price: {asian_call.mb_price_option():.2f}")
print(f"Asian Put Option Price: {asian_put.mb_price_option():.2f}")

# Binary Options
binary_call = mb_BinaryOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "call")
binary_put = mb_BinaryOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "put")

print(f"Binary Call Option Price (Cash-or-Nothing): {binary_call.mb_price_option('cash-or-nothing'):.2f}")
print(f"Binary Put Option Price (Cash-or-Nothing): {binary_put.mb_price_option('cash-or-nothing'):.2f}")

# Lookback Options
lookback_call = mb_LookbackOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "call")
lookback_put = mb_LookbackOption(spot_price, strike_price, risk_free_rate, volatility, maturity, "put")

print(f"Lookback Call Option Price: {lookback_call.mb_price_option():.2f}")
print(f"Lookback Put Option Price: {lookback_put.mb_price_option():.2f}")

# Rainbow Options
rainbow_call = mb_RainbowOption(spot_price, spot_price2, strike_price, risk_free_rate, volatility, volatility2, maturity, correlation, "call")
rainbow_put = mb_RainbowOption(spot_price, spot_price2, strike_price, risk_free_rate, volatility, volatility2, maturity, correlation, "put")

print(f"Rainbow Call Option Price: {rainbow_call.mb_price_option():.2f}")
print(f"Rainbow Put Option Price: {rainbow_put.mb_price_option():.2f}")

# Chooser Options
chooser_call = mb_ChooserOption(spot_price, strike_price, risk_free_rate, volatility, maturity, choosing_time, "call")
chooser_put = mb_ChooserOption(spot_price, strike_price, risk_free_rate, volatility, maturity, choosing_time, "put")

print(f"Chooser Option Price: {chooser_call.mb_price_option():.2f}")  # Chooser option can be interpreted as the best of both

Barrier Call Option Price: 4501.44
Barrier Put Option Price: 135.85
Asian Call Option Price: 144.00
Asian Put Option Price: 86.13
Binary Call Option Price (Cash-or-Nothing): 0.44
Binary Put Option Price (Cash-or-Nothing): 0.55
Lookback Call Option Price: 4568.06
Lookback Put Option Price: 135.54
Rainbow Call Option Price: 2681.65
Rainbow Put Option Price: 139.52
Chooser Option Price: 10.85
